In [105]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from math import log10
import pandas as pd
import numpy as np
import pymysql
import pickle

In [3]:
conn = pymysql.connect(host='127.0.0.1', user='root', passwd='210909', db='mysql', charset='utf8')
cur = conn.cursor()
cur.execute("USE trainingdb")

0

In [4]:
cur.execute('SELECT netloc,' +
            'hasTitle, hasDescription, hasH1, hasKeywords, hasViewport,' +
            'matchesInBody, keywordInDescription, keywordInH1, keywordInTitle ' +
            'from result natural join search natural join config natural join page natural join domain')
result = cur.fetchall()
cur.execute('SELECT keyword, url from result natural join search natural join config natural join page natural join domain')
header = cur.fetchall()
cur.execute('SELECT score from result natural join search natural join config natural join page natural join domain')
scores = cur.fetchall()

In [107]:
df = pd.DataFrame(list(result), columns= 'netloc, hasTitle, hasDescription, hasH1, hasKeywords, hasViewport, matchesInBody, keywordInDescription, keywordInH1, keywordInTitle'.split(', '))

In [186]:
df['score'] = pd.DataFrame(list(scores))
df['good_result'] = df['score'].apply(lambda x : 1 if x <= 5 else 0)
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
#df['netloc_id'] = pd.factorize(df['netloc'])[0]
df['matchesInBody'] = df['matchesInBody'].apply(lambda x : round(log10(x+1),1))
#df[df['netloc'] == 'en.wikipedia.org']

In [187]:
train, test = df[df['is_train'] == True], df[df['is_train'] == False]

In [209]:
features = train.columns[1:-4]
features

Index(['hasTitle', 'hasDescription', 'hasH1', 'hasKeywords', 'hasViewport',
       'matchesInBody', 'keywordInDescription', 'keywordInH1',
       'keywordInTitle'],
      dtype='object')

In [189]:
train_features = train[features]
train_features.head()

,hasTitle,hasDescription,hasH1,hasKeywords,hasViewport,matchesInBody,keywordInDescription,keywordInH1,keywordInTitle
0,1,1,1,0,1,0.5,1,1,1
1,1,1,1,0,1,0.4,1,1,1
4,1,0,0,0,1,0.3,0,0,1
5,1,1,1,0,1,0.4,0,1,1
6,1,1,1,1,1,0.4,0,1,1


In [190]:
y = train['good_result']

In [191]:
y[:10]

0     1
1     1
4     0
5     0
6     0
7     0
9     1
10    1
13    1
14    1
Name: good_result, dtype: int64

In [192]:
X_train, X_test, y_train, y_test = train_test_split(df[features], df['good_result'], test_size = 0.5)

In [193]:
clf = RandomForestClassifier(n_jobs=2, random_state=0, n_estimators=10000)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10000,
                       n_jobs=2, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [194]:
predictions = clf.predict(X_test)

In [195]:
predictions

array([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,

In [196]:
clf.predict_proba(X_test)[:10]

array([[0.7448566 , 0.2551434 ],
       [0.21328876, 0.78671124],
       [0.74582423, 0.25417577],
       [0.84110035, 0.15889965],
       [0.75768954, 0.24231046],
       [0.76466741, 0.23533259],
       [0.68098835, 0.31901165],
       [0.08289943, 0.91710057],
       [0.75768954, 0.24231046],
       [0.74582423, 0.25417577]])

In [197]:
print(accuracy_score(y_test, predictions))

0.6836734693877551


In [198]:
knn = KNeighborsClassifier()

In [199]:
knn.fit(train[features], y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [200]:
predictions = knn.predict(test[features])

In [201]:
print(accuracy_score(test['good_result'], predictions))

0.6213592233009708


In [202]:
pickle.dump(clf, open("classifier.plk", 'wb'))

In [203]:
pickle.dump(knn, open("knn-classifier.plk", 'wb'))